# Библиотеки

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
!pip install catboost

In [3]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, Ridge, HuberRegressor, ElasticNet, LinearRegression, ARDRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# Функции для обработки

In [5]:
# Для поиска выбросов будем использовать boxplot, pairplot
def get_boxplot(X, columns=None):
  if columns is None:
    columns = X.columns

  for i in columns:
    sns.boxplot(x=X[i])
    plt.show()
  pass  

def get_pairplot(X, columns=None):
  if columns is None:
    columns = X.columns

  sns.pairplot(X[columns])
  pass

def get_hist(X, columns=None, bins='auto'):
  if columns is None:
    columns = X.columns

  for i in columns:
    sns.histplot(x=X[i], bins=bins)
    plt.grid()
    plt.show()
  pass

def get_heatmap(X, columns=None):
  if columns is not None:
    X = X[columns]
    
  sns.heatmap(X.corr(), cmap='coolwarm', annot = True)
  pass

Функция выдает некоторую статистику: по каждому столбцу - количество пропусков, количество уникальных значий, тип данных.

Затем для каждого столбца, в котором число уникальных меньше лимита (50) - список уникальных значений

In [6]:
# По каждому признаку - число уникальных значений и тип
def get_stats(df, unic='all', limit=50):
  print('{0:<20} {1:>10} | {2:>10} | {3}\n'.format('Название колонки', 'Пустых', 'Уникальных', 'Тип данных')+'-'*57)
  for col in df.columns:
    print(f'{col:<20} {df[col].isnull().sum():>10} | {len(df[col].value_counts()):>10} | {df[col].dtype}')
  
  for col in df.columns:
    if len(df[col].value_counts())<limit+1:
      if unic=='object':
        if df[col].dtype=='object':
          print(f'\n{col}\n{"-"*57}')
          print(f'{df[col].value_counts()}\n{"-"*57}')
      else:
          print(f'\n{col}\n{"-"*57}')
          print(f'{df[col].value_counts()}\n{"-"*57}')          

  pass

# Добыча данных

In [5]:
path = '/content/drive/MyDrive/Авиахакатон/'
X_train_orig = pd.read_csv(path + "X_train.csv")
y_train_orig = pd.read_csv(path + "y_train.csv")
X_test_orig = pd.read_csv(path + "X_test.csv")
y_test_orig = pd.read_csv(path + "y_test.csv")

X_valid_orig = pd.read_csv(path + "X_valid.csv")

# Обработка данных

## Описательные статистики

In [8]:
get_stats(y_train_orig, limit=0)

Название колонки         Пустых | Уникальных | Тип данных
---------------------------------------------------------
flight_datetime               0 |      35772 | object
flight_phase                  0 |          2 | object
engine_id                     0 |        116 | object
BRAT                         73 |       1294 | float64
DEGT                      19515 |      27663 | float64
DELFN                     37914 |       9194 | float64
DELN1                     37914 |       9153 | float64
DELVSV                    35910 |         93 | float64
DPOIL                     28544 |       2198 | float64
EGTC                      19515 |      24780 | float64
EGTHDM                    10833 |      36793 | float64
EGTHDM_D                  28801 |      18883 | float64
GEGTMC                    28591 |      18897 | float64
GN2MC                     28591 |      18376 | float64
GPCN25                    19515 |      27220 | float64
GWFM                      19515 |      28086 | float64
PCN12  

In [9]:
get_stats(X_train_orig, limit=0)

Название колонки         Пустых | Уникальных | Тип данных
---------------------------------------------------------
engine_id                     0 |        116 | object
aircraft_id                   0 |         58 | object
flight_datetime               0 |      35772 | object
flight_phase                  0 |          2 | object
engine_position               0 |          2 | int64
n1_modifier                   0 |          8 | float64
number_blades                 0 |          3 | float64
engine_family                 0 |          3 | object
engine_type                   0 |          5 | object
manufacturer                  0 |          2 | object
ZHPTAC                    35837 |         49 | float64
ZLPTAC                    35837 |         63 | float64
ZPCN12                        0 |       2263 | float64
ZPCN25                        0 |       1622 | float64
ZPHSF                     34458 |        361 | float64
ZPHSR                     34458 |        361 | float64
ZPN12R       

## Baseline - обработка

In [6]:
X_train_orig.shape

(47794, 53)

Список столбцов с пропусками X_train 

In [7]:
null_columns = ['ZHPTAC', 'ZLPTAC', 'ZPHSF', 'ZPHSR', 'ZPN12R', 'ZPOIL', 'ZPS3', 'ZT1AB', 'ZT3', 
       'ZTAMB', 'ZTLA', 'ZTNAC', 'ZTOIL', 'ZVB1F', 'ZVB1R', 'ZVB2F', 'ZVB2R',
       'ZVSV', 'ZWF36', 'IHPSOV', 'AGW', 'CAS', 'IAI', 'IVS12', 'SAT',
       'ZVIAS', 'ZWBP1', 'ZWBP1_8E', 'ZWBP2', 'ZWBP2_8E',
       'IBP', 'IAIE']

In [8]:
# Заменяет пропуски в колонке на среднее или конкретное значение, стратегию определяет mode
def get_value(X, column, mode='mean', value=0):
  if mode == 'value':
    X.loc[X[X[column].isna()].index ,column] = value
  else:
    X.loc[X[X[column].isna()].index, column] = X[column].mean()
  return X

# Функция осущесвляет подготовку данных для расчета
def prepare_X_data(X: pd.DataFrame) -> pd.DataFrame:
  X_ref = X.copy()
  # удаляем идентификационные поля
  X_ref = X_ref.drop(columns=['engine_id', 'aircraft_id', 'flight_datetime'])
  # Заполняем пропуски в столбцах средним значением
  for col in null_columns:
    get_value(X_ref, col)

  # Конструирование признаков
  Teta = X_ref['ZT1A'] / 288.15
  P = X_ref['ZALT'] / 1013.25
  # если значения нулевые
  Teta[Teta==0] = 0.001
  P[P==0] = 10
  
  X_ref['EGTK'] = (X_ref['ZT49'] + 273.15) / Teta  
  X_ref['FFK'] = X_ref['ZWF36'] / (np.sqrt(Teta) * P )
  X_ref['N2K'] = X_ref['ZPCN25'] / np.sqrt(Teta) 
  X_ref['N1K'] = X_ref['ZPCN12'] / np.sqrt(Teta)
  X_ref.fillna(0, inplace=True)
  # Поскольку мы использовали признаки для конструирования других, исходные надо убрать
  #X_ref = X_ref.drop(columns=['ZT1A', 'ZALT', 'ZT49', 'ZWF36', 'ZPCN25', 'ZPCN12'])

  #Убираем нечисловые признаки
  #cat_features = list(X.columns[X.dtypes == object])
  #X_ref = X_ref.drop(columns=cat_features)
  

  # Категориальные признаки превращаем в фиктивные
  X_ref = pd.get_dummies(X_ref, columns=['n1_modifier', 'flight_phase', 'engine_family',
                                         'engine_type', 'manufacturer', 'aircraft_family',
                                         'aircraft_type', 'aircraft_grp', 'ac_manufacturer'])

  return X_ref

def prepare_y_data(y: pd.DataFrame) -> pd.DataFrame:
  #Убираем нечисловые признаки
  cat_features = list(y.columns[y.dtypes == object])
  y_ref = y.drop(columns=cat_features)

  # Пропуски
  #y_ref.fillna(0, inplace=True)
  return y_ref

In [9]:
X_train = prepare_X_data(X_train_orig)
y_train = prepare_y_data(y_train_orig)

X_test = prepare_X_data(X_test_orig)
y_test = prepare_y_data(y_test_orig)

X_valid = prepare_X_data(X_valid_orig)

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Стратегии, реализованные в ходе обработки данных:
- исключены идентификационные поля
- применили one-hot кодирование для категориальных признаков
- созданы новые признаки EGTK, FFK, N2K, N1K на основе технической документации
- признак n1_modifier обработан как категориальный

# Baseline-**моделирование**

Класс, осуществляющий обучение семейства моделей для каждого таргета отдельно. Выдаёт статистику по ошибкам каждой модели, определяет лучшую модель по тестовой выборке.

В итоге для каждого таргета содержит лучшую модель из семейства обученных, на которой можно строить предсказание.

В качестве набора моделей использовались как простые регрессоры из набора sklearn, так и нейросети различной архитектуры.

Идея в том, что разный таргет может быть предсказан лучше какой-то отдельной моделью, не всегда это может оказаться нейросеть или её архитектура.

In [11]:
class ModelsTraining:

  def __init__(self, targets:list, models: dict):
    self.sc = StandardScaler()
    self.targets = targets
    self.models = models
    self.predict_data = {}

  def fit(self, X, y):
    """
    Осущесвляет подгонку семейства моделей под обучающую выборку
    сохраняет в параметр fit_data словарь обученных моделей для каждого таргета
    """
    # шкалируем
    X_sc = self.sc.fit_transform(X)
    self.fit_data = {
        target: 
        {name_model:
         self.models[name_model].fit(X_sc, y[target]) if 'neuro' not in name_model 
         else 
         self.models[name_model].fit(X_sc, y[target], validation_batch_size=50,
                                     validation_data=(X_test_,y_test_),
                                     epochs=100, verbose=0, callbacks=my_callbacks) 
         for name_model in self.models
         } for target in self.targets
                    }


  def __get_best_metric__(self, target):
    return min(self.errors[target], key=self.errors[target].get)

  def get_metric(self, y_test, error: str='mse') -> pd.DataFrame:
    """
    Рассчитывает метрики для каждой модели из семейства для каждого предсказываемого таргета.
    Конкретная метрика задаётся в параметрах. В работе использовалась MSE
    Также формирует статистику для каждого таргета и модели; статистику для лучшей модели
    Возвращает metric - данные о лучшей модели и значении метрики
    """
    if error=='mse':
      err = mean_squared_error
    elif error=='mae':
      err = mean_absolute_error
    else:
      err = r2_score

    self.errors, self.metric, self.best_model = {}, {}, {}
    self.metric = {}

    for target in self.targets:
      self.errors[target] = {name_model: err(y_test[target], value) for name_model,value in self.predict_data[target].items()}
      best_model_name = self.__get_best_metric__(target)
      self.metric[target] = {'best_model': best_model_name,
                             'best_metric': self.errors[target][best_model_name]}
      self.best_model[target] = self.models[best_model_name]                      
    return pd.DataFrame(self.metric)

  def predict_models(self, X_test):
    """
    Расчет предсказанных значений на X_test для каждой из модели для каждого таргета
    """
    X_sc = self.sc.transform(X_test)
    self.predict_data = {target: 
                         {name_model: 
                          self.models[name_model].predict(X_sc) for name_model in self.models
                          } for target in self.targets
                         }
    return self.predict_data

  def predict(self, X_test) -> pd.DataFrame:
    """
    Рассчитывает и возвращает предикт на лучшей для данного таргета модели
    """
    X_sc = self.sc.transform(X_test)
    predict = pd.DataFrame([])
    for target in self.targets:
      best_model_name = self.__get_best_metric__(target)
      predict[target] = self.best_model[target].predict(X_sc).flatten()
    return predict

In [25]:
def lr_exp_decay(epoch, lr):
  """
  Управление learning_rate во время обучения.
  Для новоого таргета lr сбрасывается на начальное, по мере обучения и роста эпох уменьшается
  """
  if epoch < 2:
    lr = 0.001
  else:
    if epoch % 10 == 0:
      lr = lr /2
  return lr

optimizer = Adam(0.001, decay=1e-6)

# Прописываем условия для ранней остановки и управление LR
my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=4),
    #ModelCheckpoint(monitor='val_loss', save_best_only=True, save_weights_only=True),
    LearningRateScheduler(lr_exp_decay, verbose=0)
]

Модели нейросетей, которые наряду с простыми регрессорами участвуют в расчете.

In [26]:
model = Sequential()
model.add(Dense(50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer=optimizer)

model2 = Sequential()
model2.add(Dense(20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model2.add(Dense(10, kernel_initializer='normal', activation='relu'))
model2.add(Dense(1, kernel_initializer='normal'))
# Compile model
model2.compile(loss='mean_squared_error', optimizer=optimizer)

model3 = Sequential()
model3.add(Dense(100, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model3.add(Dense(50, kernel_initializer='normal', activation='relu'))
model3.add(Dense(1, kernel_initializer='normal'))
# Compile model
model3.compile(loss='mean_squared_error', optimizer=optimizer)

model4 = Sequential()
model4.add(Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model4.add(Dense(30, kernel_initializer='normal', activation='relu'))
model4.add(Dense(10, kernel_initializer='normal', activation='relu'))
model4.add(Dense(1, kernel_initializer='normal'))
# Compile model
model4.compile(loss='mean_squared_error', optimizer=optimizer)

In [29]:
# таргеты для предсказаний
targets = y_train.columns

# Словарь с моделями, на которых обучаемся предсказывать каждый из таргетов
models = {
    #'knn': KNeighborsRegressor(),
    'lasso': Lasso(random_state=66),
    'ridge': Ridge(random_state=66),
    #'huber': HuberRegressor(),
    #'elastic': ElasticNet(random_state=66),
    'linear': LinearRegression(),
    'ARD': ARDRegression(),
    #'tree': DecisionTreeRegressor(random_state=66),
    'random_tree': RandomForestRegressor(verbose=0, random_state=66),
    'catboost': CatBoostRegressor(iterations=50, verbose=0, random_state=66),
    'XGB': XGBRegressor(iterations=50,verbose=0, random_state=66),
    'neuro1': model,
    #'neuro2': model2,
    #'neuro3': model3,
    'neuro4': model4  
}

Цикл обучения семейства моделей на каждом из таргетов и предикт на лучшей модели

In [30]:
predict, er = pd.DataFrame([]), pd.DataFrame([])

# Перебираем каждый таргет, стратегия обучения: обучаемся только на тех значениях таргета, где нет пропусков.
# Для этого формируется X_train, y_train для якаждого таргета отдельно
# и на них производится подгонка моделей
for target in tqdm(targets):
  if target == 'VSVNOM':
    predict[target] = 0
    continue
    
  # Обучаем только на непустых данных в таргете
  y_train_ = y_train[y_train[target].notna()]
  X_train_ = X_train[y_train[target].notna()]

  y_test_ = y_test[y_test[target].notna()]
  X_test_ = X_test[y_test[target].notna()]

  # Обучение семейства моделей
  mt = ModelsTraining(targets=[target], models=models)
  mt.fit(X_train_, y_train_)

  # Вычисление метрики для каждой модели из семейства
  predicts_models = mt.predict_models(X_test_)
  # Лучшая модель и лучшая метрика
  er[target] = mt.get_metric(y_test_, error='mse')

  # На лучшей модели для таргета делаем предикт для валидационной выборки
  predict[target] = mt.predict(X_valid)

  0%|          | 0/30 [00:00<?, ?it/s]

[12:32:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
597/597 [==============================] - 1s 1ms/step


  3%|▎         | 1/30 [01:21<39:21, 81.41s/it]

[12:33:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
897/897 [==============================] - 1s 1ms/step


  7%|▋         | 2/30 [02:28<34:08, 73.17s/it]

...

 93%|█████████▎| 28/30 [29:25<02:45, 82.51s/it] 

[13:01:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
150/150 [==============================] - 0s 1ms/step


 97%|█████████▋| 29/30 [29:52<01:06, 66.32s/it]

[13:03:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
472/472 [==============================] - 1s 1ms/step


100%|██████████| 30/30 [32:15<00:00, 64.51s/it]


In [46]:
er

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,...,SLOATL,SLOATL_D,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
best_metric,0.000203,3.042057,0.552455,0.078943,0.0,0.009526,4.563458,26.300342,22.80835,7.371912,...,1.459541,1.964169,0.014516,0.000741,505.863259,0.110061,41.989501,0.00414,6.009721,9953.982406
best_model,random_tree,neuro1,catboost,catboost,ARD,ridge,random_tree,ridge,random_tree,ARD,...,neuro4,random_tree,ARD,XGB,ridge,random_tree,random_tree,random_tree,random_tree,ARD


In [47]:
er.iloc[1]

BRAT        random_tree
DEGT             neuro1
DELFN          catboost
DELN1          catboost
DELVSV              ARD
DPOIL             ridge
EGTC        random_tree
EGTHDM            ridge
EGTHDM_D    random_tree
GEGTMC              ARD
GN2MC             ridge
GPCN25            ridge
GWFM        random_tree
PCN12            linear
PCN12I           linear
PCN1AR            ridge
PCN1BR              XGB
PCN1K             ridge
PCN2C             ridge
SLOATL           neuro4
SLOATL_D    random_tree
WBE                 ARD
WBI                 XGB
WFMP              ridge
ZPCN25_D    random_tree
ZT49_D      random_tree
ZTLA_D      random_tree
ZTNAC_D     random_tree
ZWF36_D             ARD
Name: best_model, dtype: object

In [18]:
predict

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,...,SLOATL_D,VSVNOM,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
0,1.000000,400.222748,22.194572,7.362079,-0.419,-13.725376,767.586340,16.736849,-1.844130,33.771229,...,-0.542143,0.0,1.440059,0.000000,4117.388624,0.079430,-0.311250,-0.063591,-5.570,-27.831641
1,1.000000,200.930969,17.928213,5.856097,0.600,-0.921007,738.190103,43.818642,-5.442096,51.224377,...,-1.587587,0.0,1.468499,1.000000,2961.122307,0.020299,3.305498,0.069533,-9.650,2.133170
2,0.915045,-26.777582,-68.335686,7.503106,2.600,-18.299855,595.749971,84.154701,3.549748,141.206269,...,1.368788,0.0,0.947987,-0.084955,2901.352613,-0.022000,0.298004,0.062660,8.085,-23.798879
3,1.000000,11.594967,-624.843506,5.637385,-0.478,-14.348441,766.203010,-0.117500,-0.690689,35.329767,...,-0.142649,0.0,1.446231,0.000000,1804.579552,0.122727,0.602997,-0.003907,-22.150,2.324919
4,0.835083,-33.832394,-103.240227,9.882702,1.900,-22.015934,620.968973,38.558231,0.299719,91.524009,...,0.138808,0.0,1.102310,-0.164917,2957.901681,-0.023500,-2.461371,0.068597,4.995,-3.681694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28671,0.893764,-35.973972,-92.623131,5.198241,2.400,-23.624638,628.039342,82.402664,2.184437,149.589883,...,0.595557,0.0,0.925940,-0.106236,2786.925100,-0.070000,-8.683995,0.058754,12.435,-15.333152
28672,1.000000,63.956429,15.845402,5.363533,0.600,-0.955411,877.935522,52.612431,-4.490112,55.592953,...,-1.519728,0.0,1.468940,1.000000,2964.341559,-0.022895,4.568000,0.067348,-9.615,-0.682012
28673,1.000000,161.262970,142.972656,7.889599,0.200,-13.726391,767.601022,42.518867,-4.216242,34.763533,...,-0.663616,0.0,1.447775,0.000000,4116.867914,-0.003096,3.723750,-0.063592,-5.590,4.098394
28674,1.000000,132.771240,22.274544,7.618626,0.600,-0.790640,698.287607,61.251877,-3.936203,48.984623,...,-1.266148,0.0,1.452405,0.995000,2968.348549,0.059549,1.535500,0.064222,4.455,-0.121329


Сабмит

In [44]:
output_columns = list(y_train_orig.columns.drop(["flight_datetime", "flight_phase", "engine_id"]))
team_3 = pd.merge(X_valid_orig[["engine_id", "flight_datetime", "flight_phase"]], predict[output_columns], left_index=True, right_index=True)

# constModel - модель не справляется, лучше не вредить
team_3['VSVNOM'] = 0
team_3['WFMP'] = y_train_orig['WFMP'].mean()
team_3['ZWF36_D'] = y_train_orig['ZWF36_D'].mean()

team_3.to_csv(path + "y_valid_AviaNet1.csv", index=False)

In [45]:
team_3.shape

(28676, 33)